to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [3]:


## open ai api key && open source models : llama3, gemma2, mistral -- Groq (is has all models interact via api )


import os
from dotenv import load_dotenv
load_dotenv()

import openai
openai.api_key=os.getenv("OPENAI_API_KEY")

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key



In [2]:
%pip install langchain_groq

   ---------------------------------------- 0.0/103.5 kB ? eta -:--:--
   --- ------------------------------------ 10.2/103.5 kB ? eta -:--:--
   ----------- --------------------------- 30.7/103.5 kB 660.6 kB/s eta 0:00:01
   --------------------------- ----------- 71.7/103.5 kB 563.7 kB/s eta 0:00:01
   -------------------------------------- 103.5/103.5 kB 661.1 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI


## import model with passing api key 

model=ChatGroq(model="Gemma2-9b-It",groq_api_key= groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002135ED08470>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002135ED08D40>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [7]:
%pip install langchain_core

Note: you may need to restart the kernel to use updated packages.


In [10]:
from langchain_core.messages import HumanMessage, SystemMessage

messages =[
  
  SystemMessage(content="Translate the following from English to French"),
  HumanMessage(content="Helloe how are you ?")
 ]

result = model.invoke(messages) ## response then now trim it 

In [11]:
## using the string output parser 

from langchain_core.output_parsers import StrOutputParser
parser= StrOutputParser()
parser.invoke(result)

"Bonjour, comment allez-vous ? \n\n\nLet me know if you have anything else you'd like to translate! 😊\n"

In [12]:
 ## using lcl 

chain = model|parser

chain.invoke(messages)

'Bonjour, comment vas-tu ? \n'

In [13]:
## you can alsouse prompt templates 

from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language}:"

prompt= ChatPromptTemplate.from_messages(

    [("system", generic_template), ("user", "{text}")]
)



In [16]:
ans = prompt.invoke({"language": "French" , "text": "Helloe" })

In [18]:
ans.to_messages()

[SystemMessage(content='Translate the following into French:'),
 HumanMessage(content='Helloe')]

In [20]:
chain= prompt | model| parser
chain.invoke({"language": "French" , "text": "Helloe" })

'Bonjour \n'